# K-fold Cross-Validation
La k-fold cross-validation è un metodo di validazione del modello che non influisce sulla dimensione del train set.<br>
In questo notebook creeremo un modello di classificazione per l'iris dataset e utilizzeremo una 10-folds cross-validation per la validazione.
<br><br>
Importiamo le librerie necessarie.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Carichiamo l'Iris Dataset in un Dataframe.

In [3]:
iris = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", 
                   names=["sepal length","sepal width","petal length","petal width","class"])
iris.head()

sepal length  sepal width  petal length  petal width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa

E creiamo gli array numpy per addestramento e test.

In [4]:
X = iris.drop("class",axis=1).values
Y = iris["class"].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

Per eseguire una cross-validation con scikit-learn abbiamo 2 opzioni

## K-folds Cross Validation method
Utilizziamo la classe KFold di sklearn per creare 10 folds e addestrare un modello per ognuna di esse.

In [7]:
from sklearn.model_selection import KFold

lr = LogisticRegression(max_iter=200)

kfold = KFold(n_splits=10)
scores = []
              
for k, (train, test) in enumerate(kfold.split(X_train)):
    lr.fit(X_train[train], Y_train[train])
    score = lr.score(X_train[test],Y_train[test])
    scores.append(score)
    print("Fold %d: Accuracy=%.2f" %(k, score))

Fold 0: Accuracy=1.00
Fold 1: Accuracy=1.00
Fold 2: Accuracy=1.00
Fold 3: Accuracy=0.91
Fold 4: Accuracy=1.00
Fold 5: Accuracy=1.00
Fold 6: Accuracy=0.80
Fold 7: Accuracy=1.00
Fold 8: Accuracy=1.00
Fold 9: Accuracy=0.90


Possiamo calcolare l'accuracy come la media dei punteggi sui 10 modelli ottenuti tramite la cross-validation

In [8]:
accuracy = np.array(scores).mean()
print("\nValidation Accuracy = %.2f" % accuracy)


Validation Accuracy = 0.96


Un problema di questo approccio è che le classi nelle folds potrebbero essere sbilanciare<br>
(ES. su 100 esempi appartenenti ad una fold, 77 esempi appartengono alla classe positiva e 23 alla classe negativa).
<br><br>
Per risolvere questo problema possiamo utilizzare la classe <span style="font-family: Monaco">StratifiedKFold</span> che crea folds bilanciando le classi.

In [10]:
from sklearn.model_selection import StratifiedKFold

lr = LogisticRegression(max_iter=200)

kfold = StratifiedKFold(n_splits=10)
scores = []
              
for k, (train, test) in enumerate(kfold.split(X_train,Y_train)):
    lr.fit(X_train[train], Y_train[train])
    score = lr.score(X_train[test],Y_train[test])
    scores.append(score)
    print("Fold %d: Accuracy=%.2f" %(k, score))
    
print("\nValidation Accuracy = %.2f" % (np.array(scores).mean()))

Fold 0: Accuracy=1.00
Fold 1: Accuracy=1.00
Fold 2: Accuracy=1.00
Fold 3: Accuracy=0.91
Fold 4: Accuracy=1.00
Fold 5: Accuracy=0.90
Fold 6: Accuracy=1.00
Fold 7: Accuracy=0.90
Fold 8: Accuracy=1.00
Fold 9: Accuracy=1.00

Validation Accuracy = 0.97


## K-fold Cross Validation scorer
La seconda opzione che scikit-learn offre è più semplice e più efficace.<br>
E' possibile utilizzare la cross_validation come funzione di scoring passandogli il modello, questa funzione utilizza internamente la <span style="font-family: Monaco">StratifiedKFold</span> quindi le classi saranno bilanciate.

In [11]:
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(max_iter=200)
scores = cross_val_score(lr, X_train, Y_train, cv=10)

for fold,score in enumerate(scores):
    print("Fold %d score=%.4f" % (fold+1,score))
    
print("\nValidation Accuracy = %.2f" % scores.mean())

Fold 1 score=1.0000
Fold 2 score=1.0000
Fold 3 score=1.0000
Fold 4 score=0.9091
Fold 5 score=1.0000
Fold 6 score=0.9000
Fold 7 score=1.0000
Fold 8 score=0.9000
Fold 9 score=1.0000
Fold 10 score=1.0000

Validation Accuracy = 0.97


Un vantaggio di questa funzione è che, considerando che la creazione del modello per ogni fold avviene in maniera indipendente, è possibile parallelizzare il lavoro distribuendolo su più CPU.<br>
Per farlo è sufficente specificare all'interno del parametro n_jobs il numero di CPU da utilizzare.

In [12]:
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(max_iter=200)
cross_val_score(lr, X_train, Y_train, cv=10, n_jobs=7, verbose=1)

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   8 out of  10 | elapsed:    4.5s remaining:    1.1s
[Parallel(n_jobs=7)]: Done  10 out of  10 | elapsed:    4.9s finished


array([1.        , 1.        , 1.        , 0.90909091, 1.        ,
       0.9       , 1.        , 0.9       , 1.        , 1.        ])

**NOTA BENE**<br>Una volta validato il modello, utilizzando una di queste due opzioni (la seconda è più consigliata) questo va **sempre** riaddestrato sull'intero train set.

In [13]:
lr.fit(X_train, Y_train)

LogisticRegression(max_iter=200)